In [37]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import collections
import pickle as pkl

In [2]:
kroeber = pd.read_excel(f'../data/Kroeber texts.xlsx',None)
reichard = pd.read_excel(f'../data/Reichard texts.xlsx',None)
teeter = pd.read_excel(f'../data/Teeter texts.xlsx',None)

/Users/leonjdaeim/opt/miniconda3/envs/EDA/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [29]:
def processWiyotFormat(data,author,source):
    data = data[source]
    data = data[~data.apply(_isna,0).all(1)]
    alldictionaries = list()
    for i in range(0,len(data),2):
        wiyotPhrase = data.iloc[i]
        englishPhrase = data.iloc[i+1]   
        ####
        wiyotPhrase = processLine(wiyotPhrase)
        ### Do some extra processing to the english phrase
        englishPhrase = processLine(englishPhrase)
        englishPhrase = [getlowercase(x) for x in englishPhrase]
        eng2wiyotDF = pd.DataFrame([englishPhrase,wiyotPhrase]).T    
        eng2wiyotDF.columns = ['english','wiyot']
        eng2wiyotDF['author'] = author
        eng2wiyotDF['source'] = source
        alldictionaries.append(eng2wiyotDF)
    return alldictionaries

def processAuthor(data,author):
    dat_englis2wiyot = []
    for story in list(data.keys()):
        dat_englis2wiyot.append(processWiyotFormat(data,author,story))
    dat_english2wiyotDF = pd.concat(sum(dat_englis2wiyot,[]))    
    return dat_english2wiyotDF

In [33]:
kroeber_englis2wiyot = processAuthor(kroeber,'Kroeber')
reichard_englis2wiyot = processAuthor(reichard,'Reichard')
teeter_englis2wiyot = processAuthor(teeter,'Teeter')

In [35]:
allDF = pd.concat([kroeber_englis2wiyot,reichard_englis2wiyot,teeter_englis2wiyot])

In [38]:
pkl.dump(allDF,open('../results/individual_english2wiyottables.pkl','wb'))

In [ ]:
## Soft remove some sources
allDF = allDF[allDF.source!="2. Wolf's Home"] 

In [66]:
pkl.dump(allDF,open('../results/individual_english2wiyottables_clean.pkl','wb'))

In [67]:
uniqueEnglishWords = list(set(allDF.english))

In [68]:
eng2wiyot = dict()
for engWord in uniqueEnglishWords:
    try:
        """
        For each english word, get all the corresponding wiyot words and create a dictionary.
        This takes care of differences in spelling
        """
        wiyotMatch = allDF[allDF.english==engWord]['wiyot']
        wiyotMatch = list(wiyotMatch)
        wiyotMatch = list(set([_.lower() for _ in wiyotMatch]))
        eng2wiyot[engWord] = wiyotMatch
    except:
        print(engWord)

In [70]:
englishWordFreq = allDF['english'].value_counts().reset_index()
englishWordFreq.columns = ['word','frequency']

In [73]:
englishWordFreq.head()

,word,frequency
0,PST,1310
1,PROG,1299
2,3SBJ.DEF,1286
3,3SBJ.DEF.,1274
4,DEM,839


In [75]:
englishWordFreq['WiyotEquivalents'] = englishWordFreq.word.map(eng2wiyot)

In [77]:
englishWordFreq.to_csv('../results/word_single_frequency.tsv',sep='\t',index=None)

In [78]:
englishWordFreq.to_clipboard()